In [51]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier #,Ridge,Lasso,ElasticNet
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score,f1_score
from sklearn.model_selection import train_test_split,cross_val_score,KFold,RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import NearMiss
from sklearn.datasets import make_classification
import numpy as np
from collections import Counter
from sklearn.feature_selection import RFE

In [52]:
cd

C:\Users\37789


In [53]:
df = pd.read_csv('application_data.csv')

In [54]:
df.drop(columns = ['SK_ID_CURR','AMT_GOODS_PRICE'],inplace = True)

In [55]:
object_cols = df.select_dtypes(include=['object']).columns

In [56]:
df[object_cols] = df[object_cols].apply(lambda x: x.str.upper())

In [57]:
if (df['AMT_ANNUITY'].dtypes == 'int64') | (df['AMT_ANNUITY'].dtypes == 'float64'):
    print('x')

x


In [58]:
def drop_high_nulls_col(df):
    # Grouped median calculation for filling missing values
    
    for cols in df.columns:
        if (df[cols].notnull().sum() / df.shape[0]) < 0.4:
            df.drop(columns=[cols], inplace=True)
            
    return df

In [59]:
df = drop_high_nulls_col(df)

In [60]:
X_col = df.drop(columns = 'TARGET').columns
y_col = ['TARGET']

In [61]:
group_cols = ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

In [62]:
object_cols = df.select_dtypes(include=['object']).columns
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
for name in df[object_cols].columns:
    
    df[name]= label_encoder.fit_transform(df[name])
    print(df[name])

0         0
1         0
2         1
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    0
307510    0
Name: NAME_CONTRACT_TYPE, Length: 307511, dtype: int32
0         1
1         0
2         1
3         0
4         1
         ..
307506    1
307507    0
307508    0
307509    0
307510    0
Name: CODE_GENDER, Length: 307511, dtype: int32
0         0
1         0
2         1
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    0
307510    0
Name: FLAG_OWN_CAR, Length: 307511, dtype: int32
0         1
1         0
2         1
3         1
4         1
         ..
307506    0
307507    1
307508    1
307509    1
307510    0
Name: FLAG_OWN_REALTY, Length: 307511, dtype: int32
0         6
1         1
2         6
3         6
4         6
         ..
307506    6
307507    6
307508    6
307509    6
307510    6
Name: NAME_TYPE_SUITE, Length: 307511, dtype: int32
0         7
1         4
2         7
3         7
4         7
         ..
307506    7


In [63]:
X_train,X_test,y_train,y_test = train_test_split(df[X_col],df[y_col],test_size=0.3, random_state=0)

In [64]:
def null_dict(df,grouped_cols):
    imputation_dict = {}
    # Calculate the sum of null values for each column
    null_sum = df.isnull().sum()

    # Filter columns where the sum of null values is greater than 0
    columns_with_nulls = null_sum[null_sum > 0].index

    # Select only the columns from the original DataFrame that have more than 0 null values
    null_cols = df.loc[:, columns_with_nulls].columns
    for col in null_cols:
        if (df[col].dtypes == 'int64') | (df[col].dtypes == 'float64'):
            imputation_dict.update({col : df.groupby(grouped_cols)[col].median().to_dict()})
        elif (df[col].dtypes == 'O'):
            imputation_dict.update({col : df.groupby(grouped_cols)[col].agg(pd.Series.mode).to_dict()})
                
    return imputation_dict

In [65]:
imput_dict = null_dict(X_train,group_cols)

In [66]:
def fillnulls(data,grouped_cols,imput_dict):
    imputation_dict = imput_dict
    
    col_x,col_y,col_z =grouped_cols
    
    for col in imputation_dict:
        for val in imputation_dict[col]:
            val_x,val_y,val_z = val
            data.loc[(data[col_x] == val_x) & (data[col_z] == val_z) & (data[col_z] == val_z),col] = imputation_dict[col][val]
    return data

In [67]:
X_train = fillnulls(X_train,group_cols,imput_dict)
X_test = fillnulls(X_test,group_cols,imput_dict)

In [68]:
object_cols = df.select_dtypes(include=['object']).columns

In [69]:
oneh = OneHotEncoder(sparse_output=False)
oneh.fit(X_train[object_cols])

OneHotEncoder(sparse_output=False)

In [70]:
col_names = oneh.get_feature_names_out()

In [71]:
y_train.groupby(['TARGET']).size()

TARGET
0    197772
1     17485
dtype: int64

In [72]:
X_train.drop(columns = object_cols,inplace = True)
X_test.drop(columns = object_cols,inplace = True)

In [73]:
def oversample_with_adasyn(X, y, target_minority_samples=180000):
    """
    Oversample the minority class using ADASYN until the minority class reaches the target_minority_samples.
    
    Parameters:
    - X: Feature matrix
    - y: Target vector
    - target_minority_samples: Desired number of samples for the minority class after oversampling
    
    Returns:
    - X_resampled, y_resampled: Feature matrix and target vector after resampling
    """
    # Identify minority class
    classes, counts = np.unique(y, return_counts=True)
    minority_class = classes[np.argmin(counts)]
    current_minority_samples = counts[np.argmin(counts)]

    # Calculate how many new samples we need to generate for the minority class
    if current_minority_samples >= target_minority_samples:
        raise ValueError("The minority class already has more or equal samples than the target_minority_samples.")
    
    n_samples_to_generate = target_minority_samples - current_minority_samples

    # Set the sampling strategy for ADASYN
    # This will tell ADASYN to generate the specified number of samples for the minority class
    sampling_strategy = {minority_class: n_samples_to_generate}

    # Create the ADASYN instance and fit it to the data
    adasyn = ADASYN(sampling_strategy=sampling_strategy, n_neighbors=5, random_state=42)
    X_resampled, y_resampled = adasyn.fit_resample(X, y)

    return X_resampled, y_resampled

In [74]:
def undersample_with_nearmiss(X, y):
    """
    Undersample the majority class using NearMiss until it is balanced with the minority class.
    
    Parameters:
    - X: Feature matrix
    - y: Target vector
    
    Returns:
    - X_resampled, y_resampled: Feature matrix and target vector after resampling
    """
    # Identify majority and minority classes
    classes, counts = np.unique(y, return_counts=True)
    minority_class_count = min(counts)
    
    # Set the sampling strategy for NearMiss
    # This will balance the majority class with the minority class
    nm = NearMiss()
    
    # Perform the undersampling
    X_resampled, y_resampled = nm.fit_resample(X, y)
    
    # Check if undersampling was successful
    if len(np.unique(y_resampled, return_counts=True)[1]) != 2 or np.unique(y_resampled, return_counts=True)[1][0] != minority_class_count:
        raise ValueError("NearMiss did not undersample correctly.")
    
    return X_resampled, y_resampled

In [75]:
X_resampled,y_resampled = oversample_with_adasyn(X_train,y_train)

In [76]:
X_train,y_train = undersample_with_nearmiss(X_resampled,y_resampled)

In [77]:
y_train.groupby(['TARGET']).size()

TARGET
0    158757
1    158757
dtype: int64

In [78]:
scaler = StandardScaler()

In [79]:
scaler.fit(X_train)

StandardScaler()

In [80]:
X_train_scaled = scaler.transform(X_train)
X_train = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)

In [81]:
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

In [82]:
# Starting Linear Regression
logr = LogisticRegression(max_iter = 1000)
rfe = RFE(estimator=logr, n_features_to_select=70)
rfe.fit(X_train, y_train)

#Fitting Linear Regression
logr.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [83]:
X_train = X_train[rfe.get_feature_names_out()]

In [84]:
X_test = X_test[rfe.get_feature_names_out()]

In [96]:
# Starting Linear Regression
logr = LogisticRegression(max_iter = 1000)

#Fitting Linear Regression
logr.fit(X_train[rfe.get_feature_names_out()],y_train)

#Printing errors
print('Accuracy of training is '+ str(accuracy_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('Precision of training is '+ str(precision_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('Recall of training is '+ str(recall_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('F1 of training is '+ str(f1_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('ROC AUC of training is '+ str(roc_auc_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print()
print('Accuracy of test is '+ str(accuracy_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('Precision of test is '+ str(precision_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('Recall of test is '+ str(recall_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('F1 of test is '+ str(f1_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('ROC AUC of test is '+ str(roc_auc_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))

Accuracy of training is 0.8158947322007849
Precision of training is 0.8502398927291901
Recall of training is 0.7668638233274753
F1 of training is 0.8064024693157055
ROC AUC of training is 0.8158947322007849

Accuracy of test is 0.8109567064842716
Precision of test is 0.0941167014949365
Recall of test is 0.1595367847411444
F1 of test is 0.1183904559700738
ROC AUC of test is 0.5134012444326586


In [86]:
predicted_proba = logr.predict_proba(X_test[rfe.get_feature_names_out()])

In [87]:
predicted = logr.predict(X_test[rfe.get_feature_names_out()])

In [88]:
y_probs = predicted_proba[:, 1]

In [89]:
def find_best_threshold(y_true, y_probs):
    best_threshold = 0
    best_score = 0

    for threshold in np.arange(0.01, 1, 0.01):
        # Convert probabilities to binary predictions based on the current threshold
        y_pred = (y_probs >= threshold).astype(int)
        
        # Calculate the metric of interest, e.g., F1 score
        score = f1_score(y_true, y_pred)
        
        if score > best_score:
            best_score = score
            best_threshold = threshold

    return best_threshold, best_score

best_threshold, best_score = find_best_threshold(y_test['TARGET'], y_probs)

print(f"Best Threshold: {best_threshold}")
print(f"Best F1 Score: {best_score}")

Best Threshold: 0.16
Best F1 Score: 0.1505909731153163


In [95]:
# Starting Ridge
logr = RidgeClassifier(alpha=0.3)

#Fitting
logr.fit(X_train[rfe.get_feature_names_out()],y_train)

#Printing errors
print('Accuracy of training is '+ str(accuracy_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('Precision of training is '+ str(precision_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('Recall of training is '+ str(recall_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('F1 of training is '+ str(f1_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('ROC AUC of training is '+ str(roc_auc_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print()
print('Accuracy of test is '+ str(accuracy_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('Precision of test is '+ str(precision_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('Recall of test is '+ str(recall_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('F1 of test is '+ str(f1_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('ROC AUC of test is '+ str(roc_auc_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))

Accuracy of training is 0.8036181081779071
Precision of training is 0.8266945907294823
Recall of training is 0.768299980473302
F1 of training is 0.796428338230493
ROC AUC of training is 0.8036181081779071

Accuracy of test is 0.7961064018904329
Precision of test is 0.09262679357863333
Recall of test is 0.1776566757493188
F1 of test is 0.12176673825754039
ROC AUC of test is 0.513611059216252


In [91]:
cols = ['DAYS_BIRTH','REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','NAME_EDUCATION_TYPE','CODE_GENDER','DAYS_ID_PUBLISH','REG_CITY_NOT_WORK_CITY','NAME_INCOME_TYPE','FLAG_EMP_PHONE','DAYS_EMPLOYED','REG_CITY_NOT_LIVE_CITY','FLAG_DOCUMENT_3','NAME_HOUSING_TYPE','LIVE_CITY_NOT_WORK_CITY','NAME_CONTRACT_TYPE','ORGANIZATION_TYPE','FLAG_DOCUMENT_6','FLAG_WORK_PHONE','HOUR_APPR_PROCESS_START','FLAG_PHONE']